<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb -q
!wandb login

     |████████████████████████████████| 1.9 MB 33.5 MB/s 
     |████████████████████████████████| 162 kB 63.2 MB/s 
     |████████████████████████████████| 182 kB 65.7 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 162 kB 64.2 MB/s 
     |████████████████████████████████| 158 kB 69.4 MB/s 
     |████████████████████████████████| 157 kB 74.3 MB/s 
     |████████████████████████████████| 157 kB 76.0 MB/s 
     |████████████████████████████████| 157 kB 76.0 MB/s 
     |████████████████████████████████| 157 kB 75.8 MB/s 
     |████████████████████████████████| 157 kB 76.8 MB/s 
     |████████████████████████████████| 157 kB 76.0 MB/s 
     |████████████████████████████████| 157 kB 72.6 MB/s 
     |████████████████████████████████| 156 kB 81.2 MB/s 
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wand

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [4]:
tf.keras.utils.set_random_seed(123)
tf.config.experimental.enable_op_determinism()

In [5]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
vgg16 = VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3))

58889256/58889256 [==============================] - 0s 0us/step


In [6]:
PATH = '/content/drive/MyDrive/Research Internship/wear particles'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 740 files belonging to 5 classes.
Using 592 files for training.
Found 740 files belonging to 5 classes.
Using 148 files for validation.


In [7]:
class_names = train_ds.class_names
print(class_names)
num_classes = len(class_names)

['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [8]:
model_vgg16 = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  vgg16,
  layers.Flatten(),
  layers.Dense(32, activation='relu'),
  layers.Dropout(0.5), 
  layers.Dense(num_classes, activation='softmax')
])
vgg16.trainable=False
model_vgg16.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 32)                802848    
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 5)                 165       
                                                                 
Total params: 15,517,701
Trainable params: 803,013
Non-t

In [9]:
model_vgg16.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [10]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_learning", 
      name = "vgg16",
      notes = "without augment with dropout"
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vedmanivaidya. Use `wandb login --relogin` to force relogin


In [11]:
%%time
epochs=40
history = model_vgg16.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/40
19/19 [==============================] - ETA: 0s - loss: 1.4621 - accuracy: 0.4899

wandb: Adding directory to artifact (/content/wandb/run-20221018_043055-ovnecv3l/files/model-best)... Done. 0.8s


19/19 [==============================] - 93s 3s/step - loss: 1.4621 - accuracy: 0.4899 - val_loss: 0.8215 - val_accuracy: 0.7095
Epoch 2/40
19/19 [==============================] - ETA: 0s - loss: 0.9655 - accuracy: 0.5676

wandb: Adding directory to artifact (/content/wandb/run-20221018_043055-ovnecv3l/files/model-best)... Done. 0.5s


...

19/19 [==============================] - 10s 521ms/step - loss: 0.4469 - accuracy: 0.7753 - val_loss: 0.2424 - val_accuracy: 0.9189
Epoch 29/40
19/19 [==============================] - 7s 336ms/step - loss: 0.4623 - accuracy: 0.7770 - val_loss: 0.2542 - val_accuracy: 0.9257
Epoch 30/40
19/19 [==============================] - ETA: 0s - loss: 0.4601 - accuracy: 0.7872

wandb: Adding directory to artifact (/content/wandb/run-20221018_043055-ovnecv3l/files/model-best)... Done. 0.7s


19/19 [==============================] - 10s 539ms/step - loss: 0.4601 - accuracy: 0.7872 - val_loss: 0.2272 - val_accuracy: 0.9459
Epoch 31/40
19/19 [==============================] - 7s 341ms/step - loss: 0.4405 - accuracy: 0.7736 - val_loss: 0.2890 - val_accuracy: 0.9324
Epoch 32/40
19/19 [==============================] - 7s 343ms/step - loss: 0.4524 - accuracy: 0.7922 - val_loss: 0.2483 - val_accuracy: 0.9324
Epoch 33/40
19/19 [==============================] - 7s 355ms/step - loss: 0.4052 - accuracy: 0.8159 - val_loss: 0.2492 - val_accuracy: 0.9527
Epoch 34/40
19/19 [==============================] - 7s 339ms/step - loss: 0.4335 - accuracy: 0.8142 - val_loss: 0.2518 - val_accuracy: 0.9392
Epoch 35/40
19/19 [==============================] - 7s 339ms/step - loss: 0.4789 - accuracy: 0.7821 - val_loss: 0.2575 - val_accuracy: 0.9189
Epoch 36/40
19/19 [==============================] - 7s 340ms/step - loss: 0.4563 - accuracy: 0.8041 - val_loss: 0.2309 - val_accuracy: 0.9527
Epoch 37/4

wandb: Adding directory to artifact (/content/wandb/run-20221018_043055-ovnecv3l/files/model-best)... Done. 0.7s


19/19 [==============================] - 10s 533ms/step - loss: 0.4505 - accuracy: 0.7821 - val_loss: 0.2099 - val_accuracy: 0.9595
Epoch 40/40
19/19 [==============================] - 7s 337ms/step - loss: 0.4400 - accuracy: 0.8057 - val_loss: 0.2356 - val_accuracy: 0.9324
CPU times: user 2min 5s, sys: 31.4 s, total: 2min 36s
Wall time: 9min 5s


In [12]:
wandb.finish()

accuracy,▁▃▃▄▄▅▆▆▆▆▇▆▆▆▅▅▆▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇██▇███▇█
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▅▄▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▂▁▁▁▁▁▁▁▁▂▁▁▁▁▁
val_accuracy,▁▂▃▃▃▃▄▅▅▅▆▆▇▇▇▆▇▆▇▇█▇▇▇▇▇▇▇▇█▇▇█▇▇█▇▇█▇
val_loss,█▆▆▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▂▁▁▁▂▁▁▁▁▁
accuracy,0.80574
best_epoch,38
best_val_loss,0.20993
epoch,39
loss,0.43997
val_accuracy,0.93243
